# Blackjack

### 1st - Generation of the game deck

In [1]:
# Generation of the population of cards (6-deck) and also the score for each card.

import random
import numpy as np
# Population of suits and faces
suits = ["spades", "clubs", "hearts", "diamonds"]
faces = ["Two", "Three", "Four", "Five", "Six", "Seven", "Eight", "Nine", "Ten", "Jack", "Queen", "King", "Ace"]
faces_values = {"Two" : 2, "Three" : 3, "Four" : 4, "Five" : 5, "Six" : 6, "Seven" : 7, "Eight" : 8, "Nine" : 9,
                "Ten" : 10, "Jack" : 10, "Queen" : 10, "King" : 10, "Ace" : (11,1)}

# Now we iterate for both faces and suits, and create a standard deck along with a dict with its valuations.
one_deck = []
deck_values = {}
for suit in suits:
    for face in faces:
        one_deck.append(face + ' of ' + suit)
        deck_values[face + ' of ' + suit] = faces_values[face]

# As in a typical blackjack 6 decks are used to prevent card counting, a full deck is formed.

full_deck = one_deck * 6

# Preparing the final deck while remaining the full_deck untouched in case a reshuffle needs to be done
deck = full_deck.copy()
random.shuffle(deck)

### Functions to be used

In [2]:
def number_players_introduction():
    while True:
        try:
            number_players = int(input("Introduce a number of players which will play: "))
        except ValueError:
            print('Invalid number of players.')
            continue
        else:
            print ('The selected number of players is ' + str(number_players))
            print ('\n')
            break
    return number_players

In [3]:
def create_dicts_scores_wallets (num_players):
    player_list = []
    wallet = {}
    score = {}
    hand = {}
    message_new_player = 'Introduce the name for Player {}: '
    welcome_new_player = 'Hello {}, welcome to Blackjack!'
    money_new_player = '{}, please introduce now the amount of money that you will lose today: '
    for i in range(num_players):
        player_name = input(message_new_player.format(i+1))
        print(welcome_new_player.format(player_name))
        while True:
            try:
                player_money = int(input(money_new_player.format(player_name)))
            except ValueError:
                print('Invalid amount of money.')
                continue
            else:
                print ('Okay! So {}€ then!. Enjoy the game!'.format(player_money))
                print ('\n')
                break
        player_list.append(player_name)
        wallet[player_name] = player_money
        score[player_name] = 0
        hand[player_name] = []
    score['Dealer'] = 0
    hand['Dealer'] = []
    return player_list,wallet,score,hand

In [4]:
def place_bets (players):
    player_bets = {}
    for player in players:
        while True:
            try:
                player_bet = int(input('{}, you have {}€ left in your wallet. Please introduce how much money you will bet this round: '.format(player, wallet[player])))
            except ValueError:
                print('Invalid amount of money.')
                continue
            else:
                if player_bet > wallet[player]:
                    print ('The bet surpasses your amount of money left.')
                    continue
                else:
                    print ("{}'s bet is {}.".format(player, player_bet))
                    print ('\n')
                    break
        player_bets[player] = player_bet
    return player_bets

In [5]:
def is_ace (hand):
    for card in hand:
        if type(deck_values[card]) == tuple:
            return True
    return False

In [6]:
def score_update (player, hand):
    if is_ace(hand):
        hand_value = (0,0)
        count = 0
        for card in hand:
            if type(deck_values[card]) == tuple:
                if count == 0:
                    hand_value = tuple(map(sum, zip(hand_value, deck_values[card])))
                    count += 1
                else:
                    hand_value = tuple(map(sum, zip(hand_value, (1,1))))
            else:
                hand_value = tuple(map(sum, zip(hand_value, (deck_values[card],deck_values[card]))))
        if hand_value[0] > 21:
            hand_value = hand_value[1]
            # print ("{}'s hand value is {}.".format(player, hand_value))
        #else:
            # print ("{}'s hand value may be {} or {}.".format(player, hand_value[0], hand_value[1]))
    else:
        hand_value = 0
        for card in hand:
            hand_value += deck_values[card]
        # print ("{}'s hand value is {}.".format(player, hand_value))
    return hand_value


In [7]:
def blackjack(player):
    if type(score[player]) == tuple:
        if score[player][0]==21:
            return True
        else:
            return False
    else:
        return False

def check_blackjack_phase ():
    if blackjack('Dealer'):
        print('The second card of the Dealer is {}'.format(hands['Dealer'][1]))
        print('The Dealer has Blackjack! Everyone loses unless another Blackjack is in place.')
        for player in player_list:
            if blackjack(player):
                print ("{} has also got a Blackjack! This is a push.".format(player))
            else:
                print ("{} loses {}€".format(player, player_bets[player]))
                wallet[player] -= player_bets[player]
        print ('\n')
    else:
        for player in final_player_list:
            if blackjack(player):
                print ("Winner winner Chicken Dinner! {} has a Blackjack!".format(player))
                print ("{} wins {}€".format(player, round(player_bets[player]*1.5)))
                wallet[player] += round(player_bets[player]*1.5)
                print ('\n')

In [8]:
def drawing_round ():   
    print ('Cards are being dealt...')
    print ('\n')
    for n in range(2):
        for player in score:
            hands[player].append(deck.pop())
    for pl,hnd in hands.items():
        if pl == 'Dealer':
            print('The Dealer got the following first card: {}'.format(hnd[0]))
            print('\n')
            break
        print("{}'s got the following hand: {}".format(pl,hnd))
    for pl in score.keys():
        score[pl] = score_update(pl, hands[pl])

In [9]:
def draw_card(player):
    new_card = deck.pop()
    print(new_card)
    hands[player].append(new_card)
    score[player] = score_update(player, hands[player])

In [10]:
def player_round():
    for player in final_player_list:
        if blackjack(player):
            print("{} has already a Blackjack, and does not take the turn.".format(player))
            print('\n')
            continue
        else:
            print ("It's {}'s turn!".format(player))
            print ('You have bet {}€'.format(player_bets[player]))
            while True:
                print ('Your hand is now {}'.format(hands[player]))
                if type(score[player])==tuple:
                    print ("{}'s hand value may be {} or {}.".format(player, score[player][0], score[player][1]))
                else:
                    print ("Your hand's value is {}".format(score[player]))
                hit_stand = input('Dou you want to Hit(draw a new card) or Stand(hold and end your turn)? Please use (h/s) ')
                while hit_stand not in ('h','s'):
                    hit_stand = input('Please introduce a valid command: Hit (h) or Stand (s): ') 
                if hit_stand == 'h':
                    draw_card(player)
                    if type(score[player])!= tuple :
                        if score[player] > 21:
                            print ("It's a burst! You surpassed 21. You loose {}€".format(player_bets[player]))
                            wallet[player] -= player_bets[player]
                            print ('\n')
                            break
                        elif score[player] == 21:
                            print ("You reached exactly 21. Congrats! You will win if the Dealer does not reach 21!")
                            print ('\n')
                            break
                    else:
                        if score[player][0] == 21:
                            score[player] = score[player][0]
                            print ("You reached exactly 21. Congrats! You will win if the Dealer does not reach 21!")
                            print ('\n')
                            break
                else:
                    if type(score[player])!=tuple:
                        print('You stand. Your final score is {}'.format(score[player]))
                        print ('\n')
                    else:
                        print('You stand. Your final score is {}'.format(score[player][0]))
                        print ('\n')
                        score[player] = score[player][0]
                    break

In [11]:
def dealer_round():
    print("It's the turn of the Dealer.")
    print("Dealer's complete hand: {}".format(hands['Dealer']))
    if type(score['Dealer'])==tuple:
        print ("Dealer's hand value may be {} or {}.".format(score['Dealer'][0], score['Dealer'][1]))
    else:
        print ("Dealer's hand value is {}".format(score['Dealer']))
    while True:
        if type(score['Dealer'])==tuple:
            if score['Dealer'][0] > 16:
                score['Dealer'] = score['Dealer'][0]
                print ("Dealer's final hand is {}".format(hands['Dealer']))
                print ("Dealer's final score is {}".format(score['Dealer']))
                print ('\n')
                break
            else:
                draw_card('Dealer')
        else:
            if score['Dealer'] > 21:
                print ('The Dealer is busted! All remaining players win!')
                print ('\n')
                break
            elif score['Dealer'] > 16:
                print ("Dealer's final hand is {}".format(hands['Dealer']))
                print ("Dealer's final score is {}".format(score['Dealer']))
                print ('\n')
                break
            else:
                draw_card('Dealer')

In [12]:
def final_score():
    if not blackjack('Dealer'):
        if score['Dealer'] > 21:
            for player in final_player_list:
                if not blackjack(player):
                    if score[player] < 22:
                        print ("{} wins {}€".format(player, player_bets[player]))
                        wallet[player] += player_bets[player]
            print ('\n')
        else:
            for player in final_player_list:
                if not blackjack(player):
                    if score[player] < 22:
                        if score[player] < score['Dealer']:
                            print ("{} loses {}€".format(player, player_bets[player]))
                            wallet[player] -= player_bets[player]
                        elif score[player] > score['Dealer']:
                            print ("{} wins {}€".format(player, player_bets[player]))
                            wallet[player] += player_bets[player]
                        else:
                            print ("{} recovers the bet of {}€".format(player, player_bets[player]))
    print ('\n')
    print ('This is the final score!')
    final_score = score.copy()
    for pl,sc in final_score.items():
        if blackjack(pl):
            sc = 'Blackjack!'
        elif sc > 21:
            sc = 'Bust!'
        print ("{}'s final score: {}".format(pl,sc))
    print('\n')

In [13]:
def end_of_round():
    for player in player_list:
        if player not in final_player_list:
            print("{} has been eliminated".format(player))
        else:
            print("{}'s wallet has now {}€'".format(player,wallet[player]))
            if wallet[player] == 0:
                print("{} has no money left! {} is eliminated!".format(player,player))
                final_player_list.remove(player)
                del score[player]
                del hands[player]
                del player_bets[player]
    print('\n')
    print('\n')
    # Reset of all parameters just in case:
    for player in final_player_list:
        player_bets[player]=0
        score[player]=0
        hands[player]=[]
    score['Dealer']=0
    hands['Dealer']=[]
    if not final_player_list:
        play_again = 'n'
    else:
        play_again = input('Do you wanna play again? [y/n] ')
    while play_again not in ('y','n'):
        play_again = input('Please introduce a valid command: Yes(y) or No(n): ')
    if play_again == 'n':
        wannaplay = False
        print('\n')
        print ('Game Final Money:')
        for player in player_list:
            if player not in final_player_list:
                print("{}'s final wallet: 0 (-{})".format(player,initial_wallet[player]))
            else:
                if wallet[player] >= initial_wallet[player]:
                    print("{}'s final wallet: {} (+{})".format(player,wallet[player],wallet[player]-initial_wallet[player]))
                else:
                    print("{}'s final wallet: {} (-{})".format(player,wallet[player],initial_wallet[player]-wallet[player]))
        print('\n')
        print('See all you in another game of Blackjack!')
    else:
        wannaplay = True
    return wannaplay

## Main code

In [14]:
num_players = number_players_introduction()

player_list,wallet,score,hands = create_dicts_scores_wallets (num_players)

final_player_list = player_list.copy()
initial_wallet = wallet.copy()
wannaplay = True

while wannaplay:
    print('\n')
    print('A new Round Starts!')
    print('\n')
    player_bets = place_bets(final_player_list)
    drawing_round()
    check_blackjack_phase()
    if not blackjack('Dealer'):
        player_round()
        dealer_round()
    final_score()
    wannaplay = end_of_round()

Introduce a number of players which will play: 1
The selected number of players is 1


Introduce the name for Player 1: Pol
Hello Pol, welcome to Blackjack!
Pol, please introduce now the amount of money that you will lose today: 1000
Okay! So 1000€ then!. Enjoy the game!




A new Round Starts!


Pol, you have 1000€ left in your wallet. Please introduce how much money you will bet this round: 1000
Pol's bet is 1000.


Cards are being dealt...


Pol's got the following hand: ['Two of spades', 'Ace of spades']
The Dealer got the following first card: Jack of diamonds


It's Pol's turn!
You have bet 1000€
Your hand is now ['Two of spades', 'Ace of spades']
Pol's hand value may be 13 or 3.
Dou you want to Hit(draw a new card) or Stand(hold and end your turn)? Please use (h/s) h
Four of spades
Your hand is now ['Two of spades', 'Ace of spades', 'Four of spades']
Pol's hand value may be 17 or 7.
Dou you want to Hit(draw a new card) or Stand(hold and end your turn)? Please use (h/s) h
Ace of 